In [15]:
import os
import pandas as pd
import numpy as np
import rospkg

# Create a RosPack object
rospack = rospkg.RosPack()

# Get the path to the package this script is in
package_path = rospack.get_path('hri_predict_ros')

# Define the path to the logs directory
log_dir = os.path.join(package_path, 'logs')

dataframes = []

# Get all .npz files in the logs directory
npz_files = [file for file in os.listdir(log_dir) if file.endswith('.npz')]
num_files = len(npz_files) # Number of .npz files in the logs directory

# Load each .npz file and store the values in a pandas dataframe
for iter, file in enumerate(npz_files):
    print(f'Loading file {iter+1}/{num_files}')
    file_path = os.path.join(log_dir, file)
    data = np.load(file_path)
    pred_human_x = data['pred_human_x']

    dataframe = pd.DataFrame(data=pred_human_x)
    
    # TODO: fix importing files in dataframes

    pred_human_var = data['pred_human_var']
    filt_cov_mat = data['filt_cov_mat']

    # Create a pandas dataframe from the data
    # dataframe = pd.DataFrame(data={'pred_human_x': [pred_human_x],
    #                                'pred_human_var': [pred_human_var],
    #                                'filt_cov_mat': [filt_cov_mat]})

    dataframes.append(dataframe)




# Now you have a list of pandas dataframes containing the values from each .npz file

Loading file 1/22
Loading file 2/22
Loading file 3/22
Loading file 4/22
Loading file 5/22
Loading file 6/22
Loading file 7/22
Loading file 8/22
Loading file 9/22
Loading file 10/22
Loading file 11/22
Loading file 12/22
Loading file 13/22
Loading file 14/22
Loading file 15/22
Loading file 16/22
Loading file 17/22
Loading file 18/22
Loading file 19/22
Loading file 20/22
Loading file 21/22
Loading file 22/22
